In [16]:
import numpy as np
import torch
import torchvision.models as models

# define VGG16 model
VGG16 = models.vgg16(pretrained=True)

# check if CUDA is available
use_cuda = torch.cuda.is_available()

# move model to GPU if CUDA is available
if use_cuda:
    VGG16 = VGG16.cuda()
#VGG16

In [17]:
import os
from torchvision import datasets

### TODO: Write data loaders for training, validation, and test sets
## Specify appropriate transforms, and batch_sizes
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

batch_size = 20

train_transform = transforms.Compose([transforms.RandomRotation(30),
                                transforms.RandomResizedCrop(224),
                                transforms.RandomVerticalFlip(),
                                transforms.ToTensor(),
                                transforms.Normalize((0.485, 0.456, 0.406), 
                                                     (0.229, 0.224, 0.225))])

test_transform = transforms.Compose([transforms.Resize(255),
                                    transforms.CenterCrop(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.485, 0.456, 0.406), 
                                                         (0.229, 0.224, 0.225))])

train_data = datasets.ImageFolder(root="images/dogImages/train", transform=train_transform)
validation_data = datasets.ImageFolder(root="images/dogImages/valid", transform=test_transform)
test_data = datasets.ImageFolder(root="images/dogImages/test", transform=test_transform)

print(train_data)

#train_sampler = SubsetRandomSampler(np.random.shuffle(list(range(len(train_data)))))
#validation_sampler = SubsetRandomSampler(np.random.shuffle(list(range(len(validation_data)))))
test_sampler = SubsetRandomSampler(np.random.shuffle(list(range(len(test_data)))))

transform = transforms.ToTensor()

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_data, batch_size=batch_size)
#test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, sampler=test_sampler)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

Dataset ImageFolder
    Number of datapoints: 6680
    Root location: images/dogImages/train
    StandardTransform
Transform: Compose(
               RandomRotation(degrees=(-30, 30), resample=False, expand=False)
               RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
               RandomVerticalFlip(p=0.5)
               ToTensor()
               Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
           )


In [36]:
import torch.nn as nn
import torch.nn.functional as F

# define the CNN architecture
class Net(nn.Module):
    ### TODO: choose an architecture, and complete the class
    def __init__(self):
        super(Net, self).__init__()
        ## Define layers of a CNN
        # 224x224x3
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        # 112x112x16
        # If maxpool layer has a stride 4 -> 14x14x32, if maxpool stride 2 -> 56x56x32
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        # 56x56x32
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        # 28x28x64
        self.conv4 = nn.Conv2d(64, 128, 3, padding=1)
        # 14x14x128
        self.conv5 = nn.Conv2d(128, 256, 3, padding=1)
        # 7x7x256
        
        self.fc1 = nn.Linear(7*7*256, 1000)
        self.fc2 = nn.Linear(1000, 500)
        # 133 dog breeds can be found in the dataset
        self.fc3 = nn.Linear(500, 133)
        
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.25)
        self.batch_norm1 = nn.BatchNorm1d(num_features=1000)
    #    self.batch_norm2= nn.BatchNorm1d(num_features=500)
    
    def forward(self, x):
        ## Define forward behavior
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = self.pool(F.relu(self.conv5(x)))
        
        #print(x.shape)
        #return

        x = x.view(-1, 7*7*256)
        x = F.relu(self.batch_norm1(self.fc1(x)))
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        #x = F.relu(self.batch_norm2(self.fc2(x)))
        x = self.fc3(x)
        return x

#-#-# You do NOT have to modify the code below this line. #-#-#

# instantiate the CNN
model_scratch = Net()

# move tensors to GPU if CUDA is available
if use_cuda:
    model_scratch.cuda()

print(model_scratch)

Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=12544, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=500, bias=True)
  (fc3): Linear(in_features=500, out_features=133, bias=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.25, inplace=False)
)


In [37]:
import torch.optim as optim

### TODO: select loss function
criterion_scratch = nn.CrossEntropyLoss()

### TODO: select optimizer
optimizer_scratch = optim.SGD(model_scratch.parameters(), lr=0.01)

In [38]:
# the following import is required for training to be robust to truncated images
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## find the loss and update the model parameters accordingly
            ## record the average training loss, using something like
            ## train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            optimizer.zero_grad()
            output = model(data)
            #print(len(data), len(target), len(output))
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            train_loss += ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## update the average validation loss
            output = model(data)
            loss = criterion(output, target)
            valid_loss += ((1 / (batch_idx + 1)) * (loss.data - valid_loss))
        
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        ## TODO: save the model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} ==> {:.6f}). Saving model...'.format(
            valid_loss_min,
            valid_loss))
            torch.save(model.state_dict(), 'model_scratch.pt')
            valid_loss_min = valid_loss

    # return trained model
    return model

In [39]:
loaders_scratch = {'train': train_loader, 'valid': validation_loader, 'test': test_loader }
#print(loaders_scratch['train'].batch_size)
#print(loaders_scratch['valid'].batch_size)
#loaders_scratch['test'].batch_size

In [ ]:
# train the model

train(20, loaders_scratch, model_scratch, optimizer_scratch, criterion_scratch, use_cuda, 'model_scratch.pt')

# load the model that got the best validation accuracy
#model_scratch.load_state_dict(torch.load('model_scratch.pt'))

Epoch: 1 	Training Loss: 4.890132 	Validation Loss: 4.888665
Validation loss decreased (inf ==> 4.888665). Saving model...
Epoch: 2 	Training Loss: 4.888057 	Validation Loss: 4.886788
Validation loss decreased (4.888665 ==> 4.886788). Saving model...
Epoch: 3 	Training Loss: 4.885791 	Validation Loss: 4.884780
Validation loss decreased (4.886788 ==> 4.884780). Saving model...
Epoch: 4 	Training Loss: 4.883171 	Validation Loss: 4.881909
Validation loss decreased (4.884780 ==> 4.881909). Saving model...
Epoch: 5 	Training Loss: 4.878411 	Validation Loss: 4.874632
Validation loss decreased (4.881909 ==> 4.874632). Saving model...
Epoch: 6 	Training Loss: 4.868015 	Validation Loss: 4.864134
Validation loss decreased (4.874632 ==> 4.864134). Saving model...
Epoch: 7 	Training Loss: 4.861506 	Validation Loss: 4.857723
Validation loss decreased (4.864134 ==> 4.857723). Saving model...
Epoch: 8 	Training Loss: 4.855318 	Validation Loss: 4.847405
Validation loss decreased (4.857723 ==> 4.847405

In [ ]:
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(loaders['test']):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

# call test function    
test(loaders_scratch, model_scratch, criterion_scratch, use_cuda)